Data downloaded from:

https://catalog.data.gov/dataset/lottery-powerball-winning-numbers-beginning-2010/resource/056eb632-0fb0-4d32-a791-93cf74bd7429

https://catalog.data.gov/dataset/lottery-mega-millions-winning-numbers-beginning-2002

In [2]:
# Are there number combinations that have repeated?
# Create frequency table for patterns how many odds and evens
# Check for frequency of each number, additionally, check when it was last drawn

# On 10/31/17, rule change from 5/75 + 1/15 to the current 5/70 + 1/25.

### Set Up

In [2]:
import pandas as pd
import numpy as np
import datetime 

mega_raw = pd.read_csv('Lottery_Mega_Millions_Winning_Numbers__Beginning_2002.csv', encoding="Latin-1")
mega_raw = mega_raw.drop(columns=['Multiplier'])

#print(mega_raw[mega_raw['Draw Date']=='10/31/2017'].index.values)

#versions changes:
# Starting data/ White/Mega 

# 06/24/2005 -   56/46 - index 324
# 10/22/2013 -   75/15 - index 1190
# 10/31/2017 -   70/25  - index 1610

mega_first = mega_raw.loc[:323, :]
mega_second = mega_raw.loc[324:1189, :]
mega_third = mega_raw.loc[1190:1609, :]
mega_fourth = mega_raw.loc[1610:, :]

#size of each datafram
#324 rows
#866 rows
#420 rows
#262 rows

### Mega Millions Frequency 05/17/2002 - 06/21/2005

In [3]:
#mega_pre[['First','Second', 'Third', 'Fourth', 'Fifth']] = mega_pre['Winning Numbers'].str.split(' ', expand = True).astype(int) 
#mega_pre= mega_pre.drop(columns=['Winning Numbers'])
#megaball_pre_count = mega_pre['Mega Ball'].value_counts().rename_axis('Mega Ball').reset_index(name='Counts')
## calculate the last instance of the number by looping backwards
#mega_pre_main = mega_pre.drop(columns='Mega Ball')
#mega_pre_main = pd.melt(mega_pre_main, id_vars=['Draw Date'], var_name='Ball Order', value_name='Number')
#mega_pre_count = mega_pre_main['Number'].value_counts().rename_axis('White Ball').reset_index(name='Counts')


# These functions replaces the commented code aboce
def split_cols(df, name1, name2, name3, name4, name5):
    df[[name1, name2, name3, name4, name5]] = df['Winning Numbers'].str.split(' ', expand = True).astype(int) 
    df= df.drop(columns=['Winning Numbers'])
    return df   

def megaball_count(df, col_name, count_name):
    df = df[col_name].value_counts().rename_axis(col_name).reset_index(name=count_name)
    return df

def whiteball_count(df, col_name, count_name):
    df = df.drop(columns='Mega Ball')
    df = pd.melt(df, id_vars=['Draw Date'], var_name='Ball Order', value_name='Number')
    df = df['Number'].value_counts().rename_axis(col_name).reset_index(name=count_name)
    return df
    
first_df = mega_first.copy()

firstdf = split_cols(first_df, 'First', 'Second', 'Third', 'Fourth', 'Fifth')
first_mega = megaball_count(firstdf, 'Mega Ball', 'Count')
first_white = whiteball_count(firstdf, 'White Ball', 'Count')

first_white.head()
first_mega.head()

,Mega Ball,Count
0,34,14
1,3,12
2,13,12
3,21,12
4,52,11


### Mega Millions Frequency 06/24/2005 - 10/15/2013

In [4]:
second_df = mega_second.copy()

seconddf = split_cols(second_df, 'second', 'Second', 'Third', 'Fourth', 'Fifth')
second_mega = megaball_count(seconddf, 'Mega Ball', 'Count')
second_white = whiteball_count(seconddf, 'White Ball', 'Count')

second_white.head()

,White Ball,Count
0,48,99
1,2,92
2,53,89
3,20,89
4,36,89


### Mega Millions Frequency 10/22/2013 - 10/27/2017

In [5]:
third_df = mega_third.copy()

thirddf = split_cols(third_df, 'third', 'third', 'Third', 'Fourth', 'Fifth')
third_mega = megaball_count(thirddf, 'Mega Ball', 'Count')
third_white = whiteball_count(thirddf, 'White Ball', 'Count')

third_white.head()

,White Ball,Count
0,51,38
1,35,37
2,49,37
3,41,35
4,45,35


### Mega Millions Frequency 10/31/2017 - 05/01/2020

In [6]:
fourth_df = mega_fourth.copy()

fourthdf = split_cols(fourth_df, 'fourth', 'fourth', 'fourth', 'Fourth', 'Fifth')
fourth_mega = megaball_count(fourthdf, 'Mega Ball', 'Count')
fourth_white = whiteball_count(fourthdf, 'White Ball', 'Count')

fourth_mega.head()

,Mega Ball,Count
0,11,15
1,22,14
2,20,14
3,9,14
4,10,14
